<a href="https://colab.research.google.com/github/trapti-singh/Data-integration/blob/main/nlp1email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
#Build an NLP system that automatically classifies incoming business emails into intent categories to support sales and support teams

In [68]:
# Core
import pandas as pd
import numpy as np
import re

# NLP
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Download NLTK resources (run once)
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [69]:
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 113.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [70]:
import spacy
nlp = spacy.load("en_core_web_sm")


In [71]:
!pip install --upgrade pandas


In [72]:
import pandas as pd
import csv
csv.field_size_limit(2147483647) # Set to max possible integer value

df = pd.read_csv("/content/emails.csv", engine='python', on_bad_lines='skip')
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [73]:
df = df[['message']]
df.dropna(inplace=True)
print(df.shape)

(181661, 1)


In [74]:
df['message'].str.len().describe()


,message
count,1.816610e+05
mean,2.733694e+03
std,7.008063e+03
min,6.000000e+00
25%,8.600000e+02
50%,1.474000e+03
75%,2.610000e+03
max,1.013010e+06


In [75]:
df['message'].str.len().describe()


,message
count,1.816610e+05
mean,2.733694e+03
std,7.008063e+03
min,6.000000e+00
25%,8.600000e+02
50%,1.474000e+03
75%,2.610000e+03
max,1.013010e+06


In [76]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', ' ', text)          # HTML tags
    text = re.sub(r'http\S+|www\S+', ' ', text) # URLs
    text = re.sub(r'\S+@\S+', ' ', text)        # Email IDs
    text = re.sub(r'[^a-z\s]', ' ', text)       # Numbers & symbols
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [77]:
df['clean_text'] = df['message'].apply(clean_text)
df[['message', 'clean_text']].head()


,message,clean_text
0,Message-ID: <18782981.1075855378110.JavaMail.e...,message id date mon may pdt from to subject mi...
1,Message-ID: <15464986.1075855378456.JavaMail.e...,message id date fri may pdt from to subject re...
2,Message-ID: <24216240.1075855687451.JavaMail.e...,message id date wed oct pdt from to subject re...
3,Message-ID: <13505866.1075863688222.JavaMail.e...,message id date mon oct pdt from to subject mi...
4,Message-ID: <30922949.1075863688243.JavaMail.e...,message id date thu aug pdt from to subject re...


In [78]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def remove_stopwords_lemmatize(text):
    tokens = text.split()
    tokens = [
        lemmatizer.lemmatize(word)
        for word in tokens
        if word not in stop_words and len(word) > 2
    ]
    return " ".join(tokens)


In [79]:
df['processed_text'] = df['clean_text'].apply(remove_stopwords_lemmatize)
df[['clean_text', 'processed_text']].head()


,clean_text,processed_text
0,message id date mon may pdt from to subject mi...,message date mon may pdt subject mime version ...
1,message id date fri may pdt from to subject re...,message date fri may pdt subject mime version ...
2,message id date wed oct pdt from to subject re...,message date wed oct pdt subject test mime ver...
3,message id date mon oct pdt from to subject mi...,message date mon oct pdt subject mime version ...
4,message id date thu aug pdt from to subject re...,message date thu aug pdt subject hello mime ve...


In [80]:
def remove_stammering(text):
    words = text.split()
    cleaned_words = [words[0]] if words else []

    for i in range(1, len(words)):
        if words[i] != words[i-1]:
            cleaned_words.append(words[i])

    return " ".join(cleaned_words)


In [81]:
df['final_text'] = df['processed_text'].apply(remove_stammering)
df[['processed_text', 'final_text']].head()


,processed_text,final_text
0,message date mon may pdt subject mime version ...,message date mon may pdt subject mime version ...
1,message date fri may pdt subject mime version ...,message date fri may pdt subject mime version ...
2,message date wed oct pdt subject test mime ver...,message date wed oct pdt subject test mime ver...
3,message date mon oct pdt subject mime version ...,message date mon oct pdt subject mime version ...
4,message date thu aug pdt subject hello mime ve...,message date thu aug pdt subject hello mime ve...


In [82]:
df['intent'] = ""


In [83]:
INTENT_LABELS = [
    "Sales Inquiry",
    "Follow-Up",
    "Support / Issue",
    "General Query",
    "Spam / Promotional"
]


In [84]:
df.loc[0, 'intent'] = "Sales Inquiry"
df.loc[1, 'intent'] = "Follow-Up"
df.loc[2, 'intent'] = "Support / Issue"
df.loc[3, 'intent'] = "General Query"
df.loc[4, 'intent'] = "Spam / Promotional"


In [85]:
df.sample(10)[['message']]


,message
100971,Message-ID: <12982939.1075862852761.JavaMail.e...
128312,Message-ID: <32603009.1075858559894.JavaMail.e...
142565,Message-ID: <4780437.1075843027890.JavaMail.ev...
178326,Message-ID: <16400146.1075843164857.JavaMail.e...
114450,Message-ID: <16952520.1075858077716.JavaMail.e...
178572,Message-ID: <3292312.1075853408236.JavaMail.ev...
141577,Message-ID: <31231641.1075842371722.JavaMail.e...
137324,Message-ID: <25257164.1075841373167.JavaMail.e...
144740,Message-ID: <31977148.1075852243893.JavaMail.e...
145210,Message-ID: <12652019.1075842405281.JavaMail.e...


In [86]:
df['intent'].value_counts()


,count
intent,
,181656
Sales Inquiry,1
Follow-Up,1
Support / Issue,1
General Query,1
Spam / Promotional,1


In [87]:
# Create a copy of the DataFrame to ensure reliable modification
df = df.copy()

if 'intent' not in df.columns:
    df['intent'] = ""

df[['message', 'intent']].to_csv(
    "data/labeled_emails.csv",
    index=False
)

In [88]:
import pandas as pd

df = pd.read_csv("data/labeled_emails.csv")
df.head()


,message,intent
0,Message-ID: <18782981.1075855378110.JavaMail.e...,Sales Inquiry
1,Message-ID: <15464986.1075855378456.JavaMail.e...,Follow-Up
2,Message-ID: <24216240.1075855687451.JavaMail.e...,Support / Issue
3,Message-ID: <13505866.1075863688222.JavaMail.e...,General Query
4,Message-ID: <30922949.1075863688243.JavaMail.e...,Spam / Promotional


In [89]:
df.isnull().sum()
df['intent'].value_counts()


,count
intent,
Sales Inquiry,1
Follow-Up,1
Support / Issue,1
General Query,1
Spam / Promotional,1


In [95]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['intent_encoded'] = le.fit_transform(df['intent'])

df[['intent', 'intent_encoded']].head()

,intent,intent_encoded
0,Sales Inquiry,2
1,Follow-Up,0
2,Support / Issue,4
3,General Query,1
4,Spam / Promotional,3


In [96]:
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
label_mapping

{'Follow-Up': np.int64(0),
 'General Query': np.int64(1),
 'Sales Inquiry': np.int64(2),
 'Spam / Promotional': np.int64(3),
 'Support / Issue': np.int64(4),
 nan: np.int64(5)}

In [100]:
import re

def assign_intent(text):
    text = text.lower()

    if re.search(r'unsubscribe|free|offer|buy now|limited time', text):
        return 'promotional'

    if re.search(r'win money|lottery|prince|urgent transfer', text):
        return 'spam'

    if re.search(r'following up|just checking|any update', text):
        return 'follow_up'

    if re.search(r'pricing|quote|demo|purchase|contract', text):
        return 'sales'

    return 'general_query'


In [101]:
df['intent'] = df['message'].apply(assign_intent)


In [102]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['intent_encoded'] = le.fit_transform(df['intent'])


In [103]:
df['intent_encoded'].value_counts()


,count
intent_encoded,
1,125054
2,31935
3,23668
0,774
4,230


In [104]:
df['intent'].value_counts(normalize=True)


,proportion
intent,
general_query,0.688392
promotional,0.175794
sales,0.130287
follow_up,0.004261
spam,0.001266


In [105]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    df['intent_encoded'],
    test_size=0.2,
    random_state=42,
    stratify=df['intent_encoded']
)


In [107]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [108]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=20000,
        ngram_range=(1,2),
        stop_words='english'
    )),
    ('clf', LogisticRegression(
        max_iter=1000,
        class_weight='balanced'
    ))
])


In [111]:
df_sample = df.groupby('intent_encoded').sample(
    n=3000,
    random_state=42,
    replace=True
)

X = df_sample['message']
y = df_sample['intent_encoded']


In [112]:
TfidfVectorizer(
    max_features=8000,
    ngram_range=(1,1),
    min_df=5
)


TfidfVectorizer(max_features=8000, min_df=5)

In [113]:
LogisticRegression(
    solver='saga',
    max_iter=300,
    class_weight='balanced',
    n_jobs=-1
)


LogisticRegression(class_weight='balanced', max_iter=300, n_jobs=-1,
                   solver='saga')

In [114]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=8000,
        min_df=5,
        stop_words='english'
    )),
    ('clf', LogisticRegression(
        solver='saga',
        max_iter=300,
        class_weight='balanced',
        n_jobs=-1
    ))
])


In [115]:
df_sample = df.groupby('intent_encoded').sample(
    n=3000,
    random_state=42,
    replace=True
)

X = df_sample['message']
y = df_sample['intent_encoded']


In [116]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [117]:
pipeline.fit(X_train, y_train)


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=8000, min_df=5,
                                 stop_words='english')),
                ('clf',
                 LogisticRegression(class_weight='balanced', max_iter=300,
                                    n_jobs=-1, solver='saga'))])

In [118]:
y_pred = pipeline.predict(X_test)


In [119]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.89      0.94      0.91       600
           1       0.76      0.80      0.78       600
           2       0.86      0.81      0.84       600
           3       0.83      0.75      0.78       600
           4       0.96      1.00      0.98       600

    accuracy                           0.86      3000
   macro avg       0.86      0.86      0.86      3000
weighted avg       0.86      0.86      0.86      3000



In [120]:
print(confusion_matrix(y_test, y_pred))


[[566  14   4  16   0]
 [ 39 480  36  43   2]
 [ 10  56 486  36  12]
 [ 23  83  37 449   8]
 [  0   2   0   0 598]]


In [121]:
pipeline.predict([
    "Following up on our pricing discussion",
    "Limited time offer – buy now",
    "Please find the attached contract",
    "Any update on my previous email?"
])


array([0, 2, 3, 0])

In [122]:
le.inverse_transform(
    pipeline.predict([
        "Following up on our pricing discussion",
        "Limited time offer – buy now",
        "Please find the attached contract",
        "Any update on my previous email?"
    ])
)


array(['follow_up', 'promotional', 'sales', 'follow_up'], dtype=object)

In [126]:
import re

def assign_intent(text):
    text = text.lower()  # make matching case-insensitive

    if re.search(r'follow|update|checking in', text):
        return 'follow_up'
    elif re.search(r'pricing|quote|demo|purchase|contract', text):
        return 'sales'
    elif re.search(r'unsubscribe|free|offer|buy now|limited time', text):
        return 'promotional'
    elif re.search(r'win money|lottery|urgent transfer', text):
        return 'spam'
    else:
        return 'general_query'


In [127]:
df['intent'] = df['message'].apply(assign_intent)


In [128]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['intent_encoded'] = le.fit_transform(df['intent'])


In [129]:
df['intent'].value_counts()


,count
intent,
general_query,105355
follow_up,39975
sales,23067
promotional,13255
spam,9


In [124]:
TfidfVectorizer(
    max_features=12000,
    ngram_range=(1,2),
    min_df=3,
    stop_words='english'
)


TfidfVectorizer(max_features=12000, min_df=3, ngram_range=(1, 2),
                stop_words='english')

In [125]:
import joblib
joblib.dump(pipeline, "email_intent_model.joblib")
joblib.dump(le, "label_encoder.joblib")


['label_encoder.joblib']